In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from transformers import AutoModel, AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer
import json
import torch
import torch.nn as nn
import pickle

In [30]:
tokenizer = AutoTokenizer.from_pretrained('KB/bert-base-swedish-cased')

In [49]:
model = torch.load('./results/model_file_3_labels.pkl')
with open(r'./data/tokenized_test_data_arr.pkl', "rb") as input_file:
    test_data = pickle.load(input_file)
tokens = tokenizer.convert_ids_to_tokens(test_data[0]["input_ids"])
print(tokens)
dec = tokenizer.decode(test_data[0]["input_ids"])
print(dec)

['[CLS]', 'Stud', '##era', 'på', 'gymnasium', 'och', 'folkhögskola', 'När', 'vi', 'har', 'tagit', 'emot', 'din', 'ansökan', 'Om', 'du', 'har', 'lämnat', 'in', 'en', 'ansökan', 'om', 'förlängning', 'innan', 'tiden', 'för', 'ditt', 'tidigare', 'tillstånd', 'har', 'gått', 'ut', ',', 'har', 'du', 'rätt', 'att', 'vara', 'kvar', 'i', 'Sverige', 'tills', 'Migration', '##sverket', 'har', 'fattat', 'sitt', 'beslut', '.', 'Om', 'du', 'reser', 'utomlands', 'i', 'samband', 'med', 'att', 'ditt', 'tillstånd', 'går', 'ut', 'kan', 'det', 'bli', 'svårt', 'för', 'dig', 'att', 'komma', 'in', 'i', 'Sverige', 'innan', 'du', 'har', 'fått', 'ditt', 'nya', 'tillstånd', '.', 'Du', 'kan', 'då', 'behöva', 'vänta', 'på', 'ditt', 'nya', 'beslut', 'utanför', 'Sverige', '.', 'Om', 'du', 'får', 'uppehåll', '##stillstånd', 'får', 'du', 'ett', 'uppehåll', '##stillstånd', '##skort', '.', 'Kort', '##et', 'är', 'ett', 'bevis', 'på', 'att', 'du', 'har', 'tillstånd', 'att', 'vara', 'i', 'Sverige', 'och', 'innehåller', 'blan

In [50]:
# Output class
# https://huggingface.co/docs/transformers/main_classes/output#transformers.modeling_tf_outputs.TFTokenClassifierOutput

model.eval()
test_input = []
test_labels = []
test_attn = []
for i in range(len(test_data)):
    test_input.append(test_data[i]['input_ids'])
    test_labels.append(test_data[i]['labels'])
    test_attn.append(test_data[i]['attention_mask'])

print(len(test_input))
print(len(test_labels))
print(len(test_attn))
for i in range(len(test_data)):
    output = model(torch.tensor([test_data[i]['input_ids']]), attention_mask=torch.tensor([test_data[i]['attention_mask']]), labels=torch.tensor([test_data[i]['labels']]))
    print('instance loss: ', output.loss)
    print(output.logits)
    m = nn.Softmax(dim=2)
    max = m(output.logits)
    out = torch.argmax(max, dim=2)
    # print(max)
    print(out)

45
45
45
instance loss:  tensor(0.1884, grad_fn=<NllLossBackward>)
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

KeyboardInterrupt: 